In [34]:
import torch 
import torch.nn as nn 
import torchvision 
import torchvision.transforms as transforms  
import matplotlib.pyplot as plt 
from google.colab import drive
import torch.nn.functional as F
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Download MNIST datase  
train_data = torchvision.datasets.MNIST(root = '/content/drive/MyDrive/MNIST',transform=transforms.ToTensor(),train = True, download=True)
test_data = torchvision.datasets.MNIST(root = '/content/drive/MyDrive/MNIST',transform=transforms.ToTensor(),train = False)
train_data

In [48]:
#Defining hyperparameters 
input_size = 784 # 28x28
hidden_size = 500 
num_classes = 10
epochs = 3
batch_size = 100
learning_rate = 0.001



In [47]:
#Data Loader
'''Splits the training data in elements based on the batch size.''' #datapoints / batch_size = 60000 / 100 = 600 elements 
'''Each element is a list of two elements, the features and their labels. Size of each element is the specified batch size''' #100 images per element  
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size = batch_size, shuffle = True) #shuffle = True helps the model generalize
test_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size = batch_size, shuffle = False) #shuffle = False for comparing the labels against the predictions 


In [49]:
#Fully connected NN with one hidden layer 
class NN(nn.Module):
  def __init__(self,input_size, hidden_size, num_classes):
    super(NN, self).__init__()
    self.input_size = input_size
    self.l1 = nn.Linear(input_size, hidden_size) # input_size = flatten img 
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(hidden_size, num_classes) #takes as input the output(hidden_size of the previous layer)

  def forward(self,x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    # no activation and no softmax at the end 
    return out 

model = NN(input_size = input_size, hidden_size = hidden_size, num_classes = num_classes)

#Loss and optimizer 
loss_fucntion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
#Training the model 
n_total_steps  = len(train_loader)
for epoch in range(epochs): #iterates over the number of epochs 
  for i, (images, labels) in enumerate(train_loader): #iterates for each element in the train_loader 
    # original shape: [100, 1, 28 ,28] a batch of 100 gray images 28 * 28 
    # flattend shape: [100, 784]
    images = images.reshape(-1, 28*28)

    # Forward pass 
    outputs = model(images)
    loss = loss_fucntion(outputs, labels)

    #Backwards and optimize 
    optimizer.zero_grad() #empty the gradient 
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

In [54]:
#Testing the model 
#Backpropagation is not necessary on the testing phase, so don't need to compute the gradients 
with torch.no_grad():
  n_correct = 0 
  n_samples = 0 
  for images, labels in test_loader:
    images = images.reshape(-1, 28*28)
    outputs = model(images)
    # max returns (value, index)
    _, predicted = torch.max(outputs.data, 1)
    n_samples += labels.size(0) #count the number of samples 
    n_correct += (predicted == labels).sum().item() #count all the predictions that match the labels 

  acc = 100.0 * n_correct / n_samples
  print(f'Accuracy of the network on the 10000 test images: {acc} %')

Accuracy of the network on the 10000 test images: 98.585 %
